# Visual Analytics

## Assignment 3

**Instructor:** Dr. Marco D'Ambros  
**TAs:** Carmen Armenti, Mattia Giannaccari

**Contacts:** marco.dambros@usi.ch, carmen.armenti@usi.ch, mattia.giannaccari@usi.ch

**Due Date:** May 16, 2025 @ 23:55

---
The goal of this assignment is to use **Spark (PySpark)** and **Polars** in Jupyter notebooks.  
The files `trip_data.csv`, `trip_fare.csv`, and `nyc_boroughs.geojson` are available in the provided folder: [Assignment3-data](https://usi365-my.sharepoint.com/:f:/g/personal/armenc_usi_ch/Ejp7sb8QAMROoWe0XUDcAkMBoqUFk-w2Vgroup025NhAww?e=2I7SMC).

You may clean the data as needed; however, please note that specific data cleaning steps will be required in **Exercise 5**. If you choose to clean the data before Exercise 5, make sure to retain the **original dataset** for use with the Polars exercises.

- Use **Spark** to solve **Exercises 1–4**
- Use **Polars** to solve **Exercises 5–8**

You are encouraged to use [Spark window functions](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html) whenever appropriate.

Please name your notebook file as `SurnameName_Assignment3.ipynb`

## Spark

### Exercise 1
Join the `trip_data` and `trip_fare` dataframes into one and consider only data on 2013-01-01. Please specify the number of rows obtained after joining the 2 datasets.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DateType

In [2]:
session = SparkSession.builder.getOrCreate()

In [3]:
trip_data_df = session.read.option("inferSchema", "true").option("header", "true").csv("./datasets/trip_data.csv")
trip_fare_df = session.read.option("inferSchema", "true").option("header", "true").csv("./datasets/trip_fare.csv")


In [4]:
trip_data_df.select([col(c).alias(c.strip()) for c in trip_data_df.columns]) 
trip_fare_df = trip_fare_df.select([col(c).alias(c.strip()) for c in trip_fare_df.columns])

In [5]:
trip_data_df = trip_data_df.select(
    "*",
    col("pickup_datetime").cast(DateType()).alias("pickup_date")
)

In [6]:
joined_df = trip_data_df.join(trip_fare_df, on=["medallion", "hack_license", "pickup_datetime"])

In [7]:
filtered_df = joined_df.filter(joined_df.pickup_date == "2013-01-01")

In [8]:
print("Number of records for 2013-01-01: ", filtered_df.count())

Number of records for 2013-01-01:  412630


### Exercise 2
Provide a graphical representation to compare the average fare amount for trips _within_ and _across_ all the boroughs. You may want to have a look at: https://docs.bokeh.org/en/latest/docs/user_guide/topics/categorical.html#categorical-heatmaps

### Exercise 3
Consider only Manhattan, Bronx and Brooklyn boroughs. Then create a dataframe that shows the total number of trips *within* the same borough and *across* all the other boroughs mentioned before (Manhattan, Bronx, and Brooklyn) where the passengers are more or equal than 3.

For example, for Manhattan borough you should consider the total number of the following trips:
- Manhattan → Manhattan
- Manhattan → Bronx
- Manhattan → Brooklyn

You should then do the same for Bronx and Brooklyn boroughs.

### Exercise 4
Create a dataframe where each row represents a driver, and there is one column per borough.
For each driver-borough, the dataframe provides the maximum number of consecutive trips
for the given driver, within the given borough. Please consider only trips which were payed by card. 

For example, if for driver A we have (sorted by time):
- Trip 1: Bronx → Bronx
- Trip 2: Bronx → Bronx
- Trip 3: Bronx → Manhattan
- Trip 4: Manhattan → Bronx.
    
The maximum number of consecutive trips for Bronx is 2.

## Polars

### Exercise 5

Please work on the merged dataset of trips and fares and perform the following data cleaning tasks:

1. Remove trips with invalid locations (i.e. not in New York City);
3. Remove trips with invalid amounts:
    - Total amount must be greater than zero;
    - Total amount must correspond to the sum of all the other amounts.
5. Remove trips with invalid time:
    - Pick-up before drop-off;
    - Valid duration.

After each data cleaning task, report how many rows where removed. Finally report:
- Are there **duplicate trips**?
- How many trips remain after cleaning?

In [9]:
import polars as pl

In [ ]:
trip_data_df = pl.read_csv(
    "./datasets/trip_data.csv",
    infer_schema_length=100,
    has_header=True,
    encoding="us-ascii"
)
trip_fare_df = pl.read_csv(
    "./datasets/trip_fare.csv",
    infer_schema_length=100,
    has_header=True,
    schema_overrides={" tip_amount": pl.Float64},
    encoding="us-ascii"
)

In [15]:
trip_fare_df.head(5)

medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""89D227B655E5C82AECF13C3F540D4C…","""BA96DE419E711691B9445D6A6307C1…","""CMT""","""2013-01-01 15:11:48""","""CSH""",6.5,0.0,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""","""2013-01-06 00:18:35""","""CSH""",6.0,0.5,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""","""2013-01-05 18:49:41""","""CSH""",5.5,1.0,0.5,0.0,0.0,7.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""","""2013-01-07 23:54:15""","""CSH""",5.0,0.5,0.5,0.0,0.0,6.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""","""2013-01-07 23:25:03""","""CSH""",9.5,0.5,0.5,0.0,0.0,10.5


In [18]:
print("Trip data columns: ", trip_data_df.columns)
print("Trip fare columns: ", trip_fare_df.columns)

Trip data columns:  ['medallion', 'hack_license', 'vendor_id', 'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
Trip fare columns:  ['medallion', 'hack_license', 'vendor_id', 'pickup_datetime', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']


In [ ]:
trip_data_df = trip_data_df.rename({name: name.strip() for name in trip_data_df.columns})
trip_fare_df = trip_fare_df.rename({name: name.strip() for name in trip_fare_df.columns})

In [19]:
joined_df = trip_data_df.join(trip_fare_df, on=["medallion", "hack_license", "pickup_datetime"], how="inner")

In [20]:
joined_df.head(5)

medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id_right,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
str,str,str,i64,str,str,str,i64,i64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64
"""89D227B655E5C82AECF13C3F540D4C…","""BA96DE419E711691B9445D6A6307C1…","""CMT""",1,"""N""","""2013-01-01 15:11:48""","""2013-01-01 15:18:10""",4,382,1.0,-73.978165,40.757977,-73.989838,40.751171,"""CMT""","""CSH""",6.5,0.0,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",1,"""N""","""2013-01-06 00:18:35""","""2013-01-06 00:22:54""",1,259,1.5,-74.006683,40.731781,-73.994499,40.75066,"""CMT""","""CSH""",6.0,0.5,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",1,"""N""","""2013-01-05 18:49:41""","""2013-01-05 18:54:23""",1,282,1.1,-74.004707,40.73777,-74.009834,40.726002,"""CMT""","""CSH""",5.5,1.0,0.5,0.0,0.0,7.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""",1,"""N""","""2013-01-07 23:54:15""","""2013-01-07 23:58:20""",2,244,0.7,-73.974602,40.759945,-73.984734,40.759388,"""CMT""","""CSH""",5.0,0.5,0.5,0.0,0.0,6.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""",1,"""N""","""2013-01-07 23:25:03""","""2013-01-07 23:34:24""",1,560,2.1,-73.97625,40.748528,-74.002586,40.747868,"""CMT""","""CSH""",9.5,0.5,0.5,0.0,0.0,10.5


In [26]:
ny_df = joined_df.filter(
    (joined_df["pickup_latitude"] >= 40.4774) &
    (joined_df["pickup_latitude"] <= 40.9176) &
    (joined_df["pickup_longitude"] >= -74.2590) &
    (joined_df["pickup_longitude"] <= -73.7004)
)
print("Number of records before filtering: ", joined_df.shape[0])
print("Number of records after filtering: ", ny_df.shape[0])
print("Deleted records: ", joined_df.shape[0] - ny_df.shape[0])

Number of records before filtering:  14776615
Number of records after filtering:  14504034
Deleted records:  272581


In [29]:
ny_df = ny_df.filter(
    (ny_df["total_amount"] >= 0) & 
    (ny_df["total_amount"] == ny_df["fare_amount"] + ny_df["tolls_amount"] + ny_df["tip_amount"]) &
    (ny_df["pickup_datetime"] < ny_df["dropoff_datetime"])

)

### Exercise 6

Compute the **total revenue** (total_amount) grouped by:
- Pick-up hour of the day (0–23)
- Passenger count (group >=6 into “6+”)

Create a heatmap where:
- X-axis = hour
- Y-axis = passenger count group
- Cell value = average revenue per trip

### Exercise 7

Define an "anomalous trip" as one that satisfies at least two of the following:
- Fare per mile is above the 95th percentile
- Tip amount > 100% of fare
- trip_time_in_secs is less than 60 seconds but distance is more than 1 mile

Create a dataframe of anomalous trips and:
- Report how many such trips exist
- Create a scatterplot to visualize the anomaly metrics
- Describe the visualization identifying groups and outliers

### Exercise 8
For each driver (hack_license), calculate the **total profit per hour worked**, where:
> profit = 0.7 * (fare_amount + tip_amount) when the trip starts between 7:01 AM and 7:00 PM\
> profit = 0.8 * (fare_amount + tip_amount) when the trip starts between 7:01PM and 7:00 AM

Estimate "hours worked" by summing trip_time_in_secs.

Plot a line chart showing the distribution of average profit per hour **for the top 10% drivers** in terms of total trips.

Which time of day offers **best earning efficiency**?